In [1]:
import os


In [2]:
ZIPFILE="./hymenoptera_data .zip"

In [3]:
from zipfile import ZipFile

In [4]:
with ZipFile(ZIPFILE) as f:
    f.extractall()

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy

In [8]:
data_dir="./hymenoptera_data"

In [25]:
device="cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [11]:
# transform

data_transform={
    "train":transforms.Compose([transforms.RandomResizedCrop(224),
                               transforms.RandomHorizontalFlip(),
                               transforms.ToTensor()
                    ]),
    "val":transforms.Compose([transforms.Resize(256),
                               transforms.CenterCrop(224),
                               transforms.ToTensor()])
}

In [13]:
image_dataset={
    "train":datasets.ImageFolder(os.path.join(data_dir,"train")),
    
    "val":datasets.ImageFolder(os.path.join(data_dir,"val"))
}

In [14]:
dataloader={
    "train":torch.utils.data.DataLoader(image_dataset["train"], batch_size=4, shuffle=True,num_workers=4),
    
    "val":torch.utils.data.DataLoader(image_dataset["val"], batch_size=4, shuffle=True,num_workers=4)
}

In [16]:
data_sizes={
    "train":len(image_dataset["train"]),
    "val":len(image_dataset["val"])
}
data_sizes

{'train': 244, 'val': 153}

In [17]:
class_names=image_dataset["train"].classes

In [18]:
class_names

['ants', 'bees']

In [19]:
model_conv=torchvision.models.resnet18(pretrained=True)
print(model_conv)
#freezing the conv layers
#for param in model_conv

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\kulki/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100.0%


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [20]:
for param in model_conv.parameters():
    param.required_grad=False

In [21]:
num_features=model_conv.fc.in_features
num_features

512

In [22]:
no_of_classes=len(class_names)
model_conv.fc=nn.Linear(num_features, no_of_classes)

In [24]:
print(model_conv)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [26]:
model_conv=model_conv.to(device)

In [27]:
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.Adam(model_conv.fc.parameters(), lr=0.001)

In [28]:
exp_lr_scheduler=lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
def train_loop(model, loss_fn, optimizer, scheduler, num_epochs=20):
    
    best_models_wts=copy.deepcopyy(model.state_dict())
    best_acc=0.0
    for epoch in range(num_epochs):
        print(f"Epochs {epoch}/{num_epochs -1}")
        print("**"*10)
        
        for phase in ["train","val"]:
            training= phase=="train"
            if training:
                model.train()# Sets the training modes
            else:
                model.eval()# sets the evaluations modes
            running_loss=0.0
            running_corrects=0
            
            for inputs, labels in dataLoader[phase]:
                inputs=inputs.to(device)
                labels=labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(training):
                    outputs=model(inputs)
                    _,preds=torch.max(outputs, 1)
                    loss=loss_fn(outputs, preds)
                    
                    
                    if training:
                        loss.backword()
                        optimizer.step()
        
    